In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
try:
  %tensorflow_version 1.x
except Exception:
  pass

import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

In [0]:
import numpy as np
np.random.seed(7)

In [0]:
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [0]:
latent_dimension = 100
noise_shape = (latent_dimension,)

In [0]:
image_shape = (X_train.shape[1], X_train.shape[2], 1)

In [0]:
print(X_train.shape, X_test.shape)

In [0]:
X_train = (X_train.astype('float32') - 127.5) / 127.5
X_test = (X_test.astype('float32') - 127.5) / 127.5

In [0]:
X_train = np.expand_dims(X_train, axis=3)
X_test = np.expand_dims(X_test, axis=3)

In [0]:
print(X_train.shape, X_test.shape)

In [0]:
from keras.optimizers import Adam

optimizer = Adam(lr=0.0001)

In [0]:
from keras.optimizers import SGD

optimizer = SGD(lr=0.0005, momentum=0.9, nesterov=True)

In [0]:
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Dense
from keras.layers import Dropout, Flatten, Reshape, BatchNormalization

from keras.layers.core import Activation

from keras.models import Sequential, Model

from keras.layers.advanced_activations import LeakyReLU

In [0]:
discriminator = Sequential(name='discriminator')

discriminator.add(Conv2D(64, (5, 5), padding='same', input_shape=image_shape))
discriminator.add(Activation('tanh'))
discriminator.add(MaxPooling2D(pool_size=(2, 2)))

discriminator.add(Conv2D(128, (5, 5)))
discriminator.add(Activation('tanh'))
discriminator.add(MaxPooling2D(pool_size=(2, 2)))

discriminator.add(Flatten())

discriminator.add(Dense(1024))
discriminator.add(Activation('tanh'))

discriminator.add(Dense(1))
discriminator.add(Activation('sigmoid'))

discriminator.summary()

discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [0]:
generator_shape = (7, 7, 128)
generator_size = np.prod(generator_shape)

In [0]:
generator = Sequential(name='generator')

generator.add(Dense(1024, input_shape=noise_shape))    
generator.add(Activation('tanh'))

generator.add(Dense(generator_size))    
generator.add(BatchNormalization())
generator.add(Activation('tanh'))

generator.add(Reshape(generator_shape))

generator.add(UpSampling2D(size=(2, 2)))
generator.add(Conv2D(64, (5, 5), padding='same'))
generator.add(Activation('tanh'))

generator.add(UpSampling2D(size=(2, 2)))
generator.add(Conv2D(1, (5, 5), padding='same'))
generator.add(Activation('tanh'))

generator.summary()

generator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [0]:
gan = Sequential(name='gan')
gan.add(generator)
discriminator.trainable = False
gan.add(discriminator)
gan.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [0]:
import matplotlib.pyplot as plot

def plot_generated(number_of_samples=10, dim=(1, 10), figsize=(12, 2)):

    noise = np.random.uniform(-1, 1, size=(number_of_samples, latent_dimension))
    generated_images = generator.predict(noise)
    generated_images = generated_images*127.5 + 127.5
    generated_images = generated_images.reshape(number_of_samples, image_shape[0], image_shape[1])

    plot.figure(figsize=figsize)
    for i in range(number_of_samples):
        plot.subplot(dim[0], dim[1], i+1)
        plot.imshow(generated_images[i], interpolation='nearest', cmap='gray_r')
        plot.axis('off')

    plot.tight_layout()
    plot.show()

In [0]:
import matplotlib.pyplot as plot

def plot_loss(losses):

    discriminator_loss = [v[0] for v in losses["discriminator"]]
    generator_loss = [v[0] for v in losses["generator"]]
    
    plot.figure(figsize=(10,8))
    plot.plot(discriminator_loss, label="Discriminator loss")
    plot.plot(generator_loss, label="Generator loss")

    plot.xlabel('Epochs')
    plot.ylabel('Loss')

    plot.legend()
    plot.show()

In [0]:
losses = {"discriminator":[], "generator":[]}

def train(epochs, batch_size=128, plot_frequency=1):    

    half_batch = int(batch_size / 2)
       
    for epoch in range(epochs):  

        ##########################################################################################
        # Train discriminator        
        ##########################################################################################

        # Select a random half batch of images
        indices = np.random.randint(0, X_train.shape[0], size=half_batch)
        images = X_train[indices]        

        # Generate a half batch of images          
        noise = np.random.uniform(-1, 1, size=(half_batch, latent_dimension))  
        generated_images = generator.predict(noise)
        
        # Train discriminator
        discriminator.trainable = True
        discriminator_loss_real = discriminator.train_on_batch(images, np.ones((half_batch, 1)))
        discriminator_loss_fake = discriminator.train_on_batch(generated_images, np.zeros((half_batch, 1)))
        discriminator_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)
        ##########################################################################################


        ##########################################################################################
        # Train generator  
        ##########################################################################################      

        noise = np.random.normal(0, 1, (batch_size, latent_dimension))

        # The generator wants the discriminator to label 
        # the generated samples as valid
        valid_y = np.array([1] * batch_size)

        # Train generator 
        discriminator.trainable = False
        generator_loss = gan.train_on_batch(noise, valid_y)
        ##########################################################################################        

        if (epoch > 0)  and (epoch%plot_frequency == 0):
            losses["discriminator"].append(discriminator_loss)
            losses["generator"].append(generator_loss)
            
            plot_generated()

    plot_loss(losses)

In [0]:
train(30000, batch_size=128, plot_frequency=200)